In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
jsonFile = '2019/2019schedule.json'

In [3]:
curSch = pd.read_json(jsonFile)
# curSch = curSch[['week','away_team','away_points','home_points','home_team',
#                  'away_conference','home_conference','neutral_site']]

In [4]:
curSch.loc[curSch.away_team == 'Notre Dame','away_conference'] = 'Notre Dame'
curSch.loc[curSch.home_team == 'Notre Dame','home_conference'] = 'Notre Dame'

In [5]:
FBSteams = curSch['home_team'].unique()
FBSteams.sort()

In [6]:
P5 = ['ACC','Pac-12','Big Ten','Big 12','SEC','Notre Dame']
G5 = ['Conference USA','Sun Belt','Mid-American','American Athletic','Mountain West','FBS Independents']

In [7]:
FBSdict = {}
for team in FBSteams:
    FBSdict[team] = curSch[(curSch['away_team'] == team) | 
                           (curSch['home_team'] == team)]
    FBSdict[team] = FBSdict[team][np.isfinite(FBSdict[team].away_points)]
    FBSdict[team] = FBSdict[team].reset_index()

In [8]:
def seasonProgression(dfR):
    totalGames = len(dfR)
    curWeek = int(dfR['week']) + 1
    return curWeek/(2*totalGames)

In [9]:
def winningTeam(dfR):
    if dfR['home_points'] > dfR['away_points']:
        return dfR['home_team']
    else:
        return dfR['away_team']
    
def teamWin(dfR, team):
    if dfR.winner == team:
        return True
    else:
        return False

In [10]:
def findOpp(dfR):
    if dfR.home == True:
        return dfR.away_team
    else:
        return dfR.home_team
def findOppConf(dfR):
    if dfR.home == True:
        return dfR.away_conference
    else:
        return dfR.home_conference

In [11]:
def locMult(dfR):
    if dfR['neutral_site'] == True:
        return 1.05
    elif dfR['home'] == True:
        return 1.1
    else:
        return 1.

In [12]:
def pointDiff(dfR,team):
    if dfR.home_team == team:
        return dfR.home_points - dfR.away_points
    else:
        return dfR.away_points - dfR.home_points

In [13]:
def confMult(dfR):
    if dfR.opp_conference in P5:
        return 1.
    elif dfR.opp_conference in G5:
        return 0.5
    else:
        return 0.15

In [14]:
def winQ(dfR,week): 
    if dfR.opp_conference == None:
        oppWinPct = 0.5
    else:
#         print(dfR.opp_conference)
        oppWinPct = FBSdict[dfR.opp].loc[FBSdict[dfR.opp].week <= week, 'winPct'].tail(1).item()
    
    if dfR.teamWin == True:
        winMult = 1
        oppFactor = oppWinPct
    else:
        winMult = -1
        oppFactor = 1 - oppWinPct
    
    return oppFactor * dfR.seasonProg * dfR.conf_mult * winMult*(dfR.loc_mult + 0.01*dfR.point_diff)

***


In [16]:
FBSdict['Rutgers']['seasonProg'] = FBSdict['Rutgers'].apply(lambda row: seasonProgression(row), axis = 1)
FBSdict['Rutgers']['winner'] = FBSdict['Rutgers'].apply(lambda row: winningTeam(row), axis = 1)

In [17]:
wins = 0
losses = 0
winList = []
lossList = []
for row in FBSdict['Rutgers'].itertuples():
#     print(row.winner)
    if row.winner == 'Rutgers':
        wins += 1
    else:
        losses += 1
    winList.append(wins)
    lossList.append(losses)

In [18]:
winList, lossList
FBSdict['Rutgers']['wins'] = winList
FBSdict['Rutgers']['losses'] = lossList

In [19]:
hometeam = FBSdict['Rutgers']['home_team'] == 'Rutgers'
FBSdict['Rutgers']['home'] = hometeam

In [20]:
FBSdict['Rutgers']['winPct'] = FBSdict['Rutgers']['wins']/(FBSdict['Rutgers']['losses'] + FBSdict['Rutgers']['wins'])

In [21]:
def findOpp(dfR):
    if dfR.home == True:
        return dfR.away_team
    else:
        return dfR.home_team
def findOppConf(dfR):
    if dfR.home == True:
        return dfR.away_conference
    else:
        return dfR.home_conference

In [22]:
FBSdict['Rutgers']['opp'] = FBSdict['Rutgers'].apply(lambda row: findOpp(row), axis = 1)
FBSdict['Rutgers']['opp_conference'] = FBSdict['Rutgers'].apply(lambda row: findOppConf(row), axis = 1)

In [23]:
def locMult(dfR):
    if dfR['neutral_site'] == True:
        return 1.05
    elif dfR['home'] == True:
        return 1.1
    else:
        return 1.

In [24]:
def pointDiff(dfR,team):
    if dfR.home_team == team:
        return dfR.home_points - dfR.away_points
    else:
        return dfR.away_points - dfR.home_points

In [25]:
FBSdict['Rutgers']['point_diff'] = FBSdict['Rutgers'].apply(lambda row: pointDiff(row,'Rutgers'), axis = 1)

In [26]:
FBSdict['Rutgers']['loc_mult'] = FBSdict['Rutgers'].apply(lambda row: locMult(row), axis = 1)

In [27]:
def confMult(dfR):
    if dfR.opp_conference in P5:
        return 1.
    elif dfR.opp_conference in G5:
        return 0.5
    else:
        return 0.15

In [28]:
FBSdict['Rutgers']['conf_mult'] = FBSdict['Rutgers'].apply(lambda row: confMult(row), axis = 1)

In [33]:
FBSdict['Rutgers'].winPct

0     1.000000
1     0.500000
2     0.333333
3     0.250000
4     0.200000
5     0.166667
6     0.142857
7     0.250000
8     0.222222
9     0.200000
10    0.181818
11    0.166667
Name: winPct, dtype: float64

In [36]:
FBSdict['Rutgers'].loc[FBSdict['Rutgers'].week <= 14, 'winPct'].tail(1).iloc[0]

0.16666666666666666

In [ ]:
FBSdict['UMass']['week']

In [ ]:
FBSdict['Akron']['win_q'] = FBSdict['Akron'].apply(lambda row: winQ(row,14),axis = 1)

In [130]:
FBSdict['Alabama']

,index,attendance,away_conference,away_line_scores,away_points,away_post_win_prob,away_team,conference_game,home_conference,home_line_scores,...,wins,losses,home,winPct,opp,opp_conference,point_diff,loc_mult,conf_mult,win_q_14
0,45,NaN,ACC,"[0, 3, 0, 0]",3.0,0.000009,Duke,False,SEC,"[0, 14, 21, 7]",...,1,0,True,1.000000,Duke,ACC,39.0,1.05,1.00,0.028571
1,120,NaN,FBS Independents,"[0, 0, 7, 3]",10.0,0.000006,New Mexico State,False,SEC,"[21, 17, 24, 0]",...,2,0,True,1.000000,New Mexico State,FBS Independents,52.0,1.10,0.50,0.009643
2,178,NaN,SEC,"[14, 10, 10, 13]",47.0,0.930849,Alabama,True,SEC,"[10, 0, 3, 10]",...,3,0,False,1.000000,South Carolina,SEC,24.0,1.00,1.00,0.039365
3,239,NaN,Conference USA,"[0, 7, 0, 0]",7.0,0.000014,Southern Mississippi,False,SEC,"[14, 14, 14, 7]",...,4,0,True,1.000000,Southern Mississippi,Conference USA,42.0,1.10,0.50,0.052778
4,312,NaN,SEC,"[10, 0, 7, 14]",31.0,0.000187,Ole Miss,True,SEC,"[7, 31, 14, 7]",...,5,0,True,1.000000,Ole Miss,SEC,28.0,1.10,1.00,0.065714
5,410,NaN,SEC,"[14, 10, 10, 13]",47.0,0.986383,Alabama,True,SEC,"[7, 6, 7, 8]",...,6,0,False,1.000000,Texas A&M,SEC,19.0,1.00,1.00,0.132222
6,498,NaN,SEC,"[7, 3, 3, 0]",13.0,0.002381,Tennessee,True,SEC,"[14, 7, 7, 7]",...,7,0,True,1.000000,Tennessee,SEC,22.0,1.10,1.00,0.165000
7,547,NaN,SEC,"[0, 0, 0, 7]",7.0,0.000004,Arkansas,True,SEC,"[17, 24, 7, 0]",...,8,0,True,1.000000,Arkansas,SEC,41.0,1.10,1.00,0.059921
8,629,NaN,SEC,"[10, 23, 0, 13]",46.0,0.948971,LSU,True,SEC,"[7, 6, 7, 21]",...,8,1,True,0.888889,LSU,SEC,-5.0,1.10,1.00,-0.000000
9,669,NaN,SEC,"[21, 14, 3, 0]",38.0,0.996396,Alabama,True,SEC,"[7, 0, 0, 0]",...,9,1,False,0.900000,Mississippi State,SEC,31.0,1.00,1.00,0.202738


In [30]:
FBSdict['Rutgers'].win_q.mean()

AttributeError: 'DataFrame' object has no attribute 'win_q'

In [31]:
FBSdict['Rutgers'].tail(1).wins.item(),FBSdict['Rutgers'].tail(1).losses.item()

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `item` has been deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


(2, 10)